In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('master_instruments2.csv')

In [2]:
df["ExpiryDate"].unique()   

array(['2025-03-27 14:30:00', '2025-04-24 14:30:00',
       '2025-02-27 14:30:00'], dtype=object)

In [21]:
import re

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, second expiry contract description)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Check if the first future contract exists in the DataFrame
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]

        if not fut1_df.empty:
            fut1_exchange_id = fut1_df['ExchangeInstrumentID'].values[0]  # Extract ExchangeInstrumentID safely
        else:
            fut1_exchange_id = None  # Handle missing case

        fut2 = f"{stock_symbol}{expiry2}FUT"

        return fut1_exchange_id, fut2
    else:
        return None

# Iterate over descriptions ending with "SPD"
for i in df[df["Strike"].str.endswith("SPD")]['Description']:
    result = extract_futures(i, df)

    if result:
        print("Extracted Futures:")
        print(result[0] if result[0] is not None else "ExchangeInstrumentID not found")  # TITAN25MARFUT ExchangeInstrumentID or None
        print(result[1])  # TITAN25APRFUT
    else:
        print(f"No match found for {i}")


Extracted Futures:
41463
TECHM25MARFUT
Extracted Futures:
43400
TITAN25APRFUT
Extracted Futures:
41029
ITC25APRFUT
Extracted Futures:
35057
ADANIENT25APRFUT
Extracted Futures:
43438
WIPRO25APRFUT
Extracted Futures:
41559
WIPRO25MARFUT
Extracted Futures:
43396
TECHM25APRFUT
Extracted Futures:
35033
ADANIENT25MARFUT
Extracted Futures:
41513
ULTRACEMCO25MARFUT
Extracted Futures:
39122
JSWSTEEL25MARFUT
Extracted Futures:
35215
ASIANPAINT25APRFUT
Extracted Futures:
39039
ITC25MARFUT
Extracted Futures:
35245
AXISBANK25APRFUT
Extracted Futures:
41615
JSWSTEEL25APRFUT
Extracted Futures:
35310
AXISBANK25MARFUT
Extracted Futures:
43406
ULTRACEMCO25APRFUT
Extracted Futures:
35154
ASIANPAINT25MARFUT
Extracted Futures:
41498
TITAN25MARFUT


In [23]:
import re
import json

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, ExchangeInstrumentID of second expiry)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Fetch F1 ExchangeInstrumentID
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]
        fut1_exchange_id = int(fut1_df['ExchangeInstrumentID'].values[0]) if not fut1_df.empty else None

        # Fetch F2 ExchangeInstrumentID
        fut2_df = df[df['Description'] == f"{stock_symbol}{expiry2}FUT"]
        fut2_exchange_id = int(fut2_df['ExchangeInstrumentID'].values[0]) if not fut2_df.empty else None

        return fut1_exchange_id, fut2_exchange_id
    else:
        return None, None

# Dictionary to store results
futures_mapping = {}

# Iterate over descriptions ending with "SPD"
for _, row in df[df["Strike"].str.endswith("SPD")].iterrows():
    parent_exchange_id = int(row['ExchangeInstrumentID'])  # Get parent ExchangeInstrumentID
    result = extract_futures(row['Description'], df)

    if result:
        f1_exchange_id = result[0]
        f2_exchange_id = result[1]
        
        # Store only if both F1 and F2 are found
        if f1_exchange_id and f2_exchange_id:
            futures_mapping[parent_exchange_id] = [f1_exchange_id, f2_exchange_id]

# Save to JSON file
output_file = r"D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json"
with open(output_file, "w") as json_file:
    json.dump(futures_mapping, json_file, indent=4)

print(f"Futures mapping saved at: {output_file}")


Futures mapping saved at: D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json


In [25]:
import re
import json

def extract_futures(symbol, df):
    """
    Extracts future contract details from the given symbol.

    Args:
    symbol (str): The future contract description.
    df (DataFrame): The DataFrame containing futures contract data.

    Returns:
    tuple: (ExchangeInstrumentID of first expiry, ExchangeInstrumentID of second expiry)
    """
    # Regular expression to match stock symbol and expiry months
    pattern = re.match(r"([A-Z\&\-]+)(\d{2}[A-Z]+)(\d{2}[A-Z]+)FUT", symbol)
    
    if pattern:
        stock_symbol = pattern.group(1)  # Extract stock symbol (e.g., TITAN)
        expiry1 = pattern.group(2)       # First expiry (e.g., 25MAR)
        expiry2 = pattern.group(3)       # Second expiry (e.g., 25APR)

        # Fetch F1 ExchangeInstrumentID
        fut1_df = df[df['Description'] == f"{stock_symbol}{expiry1}FUT"]
        fut1_exchange_id = int(fut1_df['ExchangeInstrumentID'].values[0]) if not fut1_df.empty else None

        # Fetch F2 ExchangeInstrumentID
        fut2_df = df[df['Description'] == f"{stock_symbol}{expiry2}FUT"]
        fut2_exchange_id = int(fut2_df['ExchangeInstrumentID'].values[0]) if not fut2_df.empty else None

        return fut1_exchange_id, fut2_exchange_id
    else:
        return None, None

# Dictionary to store parent-child mapping
futures_mapping = {}

# List to store ExchangeInstrumentIDs
exchange_instrument_list = []

# Iterate over descriptions ending with "SPD"
for _, row in df[df["Strike"].str.endswith("SPD")].iterrows():
    parent_exchange_id = int(row['ExchangeInstrumentID'])  # Convert to int
    result = extract_futures(row['Description'], df)

    if result:
        f1_exchange_id = result[0]
        f2_exchange_id = result[1]

        # Store only if both F1 and F2 are found
        if f1_exchange_id and f2_exchange_id:
            futures_mapping[parent_exchange_id] = [f1_exchange_id, f2_exchange_id]

            # Add all IDs to the list
            exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": f1_exchange_id})
            exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": f2_exchange_id})

    # Add parent exchangeInstrumentID to the list as well
    exchange_instrument_list.append({"exchangeSegment": 2, "exchangeInstrumentID": parent_exchange_id})

# Remove duplicates from the list
exchange_instrument_list = list({v["exchangeInstrumentID"]: v for v in exchange_instrument_list}.values())

# Save Futures Mapping to JSON
futures_mapping_file = r"D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json"
with open(futures_mapping_file, "w") as json_file:
    json.dump(futures_mapping, json_file, indent=4)

# Save ExchangeInstrumentID List to JSON
exchange_instrument_file = r"D:\YASH\Cleaning\Data\Futuresdataforwardfil\exchange_instruments.json"
with open(exchange_instrument_file, "w") as json_file:
    json.dump(exchange_instrument_list, json_file, indent=4)

print(f"✅ Futures mapping saved at: {futures_mapping_file}")
print(f"✅ Exchange Instrument List saved at: {exchange_instrument_file}")


✅ Futures mapping saved at: D:\YASH\Cleaning\Data\Futuresdataforwardfil\futures_mapping.json
✅ Exchange Instrument List saved at: D:\YASH\Cleaning\Data\Futuresdataforwardfil\exchange_instruments.json


In [3]:
df[df["Name"]=="TITAN"]

,Unnamed: 0,ExchangeSegment,ExchangeInstrumentID,Name,Description,Series,NameWithSeries,InstrumentID,PriceBand.High,PriceBand.Low,FreezeQty,TickSize,LotSize,Multiplier,displayName,ExpiryDate,Strike,OptionType,TickerName
2,2,NSEFO,73632,TITAN,TITAN25APRFUT,FUTSTK,TITAN-FUTSTK,2511400073632,3907.00,3196.65,7001,0.05,175,1,TITAN,2025-04-24 14:30:00,TITAN 24APR2025,1,1
5,5,NSEFO,43400,TITAN,TITAN25MARFUT,FUTSTK,TITAN-FUTSTK,2508600043400,3881.35,3175.65,7001,0.05,175,1,TITAN,2025-03-27 14:30:00,TITAN 27MAR2025,1,1
24,24,NSEFO,41498,TITAN,TITAN25FEBFUT,FUTSTK,TITAN-FUTSTK,2505800041498,3858.80,3157.20,7001,0.05,175,1,TITAN,2025-02-27 14:30:00,TITAN 27FEB2025,1,1
28,28,NSEFO,11184032,TITAN,TITAN25MAR25APRFUT,FUTSTK,TITAN-FUTSTK,2100111184032,88.25,-88.25,7001,0.05,175,1,NaN,2025-03-27 14:30:00,TITAN 27MAR24APR SPD,1,1
44,44,NSEFO,10666888,TITAN,TITAN25FEB25MARFUT,FUTSTK,TITAN-FUTSTK,2100110666888,87.70,-87.70,7001,0.05,175,1,NaN,2025-02-27 14:30:00,TITAN 27FEB27MAR SPD,1,1


In [26]:
#create a dic of ExchangeInstrumentID and  Description

df1=df[["ExchangeInstrumentID","Description"]]
df1=df1.drop_duplicates()
df1=df1.set_index("ExchangeInstrumentID")
df1=df1.to_dict()
#save to json
import json
with open('ExchangeInstrumentID.json', 'w') as fp:
    json.dump(df1, fp)
    

In [17]:
import re

def extract_symbols(text):


    matches = re.findall(r'^([A-Z]+)\s\d{2}([A-Z]+)\d{2}([A-Z]+)\sSPD$', text)
    
    if matches:

        stock_name, month1, month2 = matches[0]
        symbol1 = f"{stock_name}{month1}"
        symbol2 = f"{stock_name}{month2}"
        return symbol1, symbol2
    else:
        return None
spdsym=df[df["displayName"].isna()]["Description"].apply(extract_symbols)
print(spdsym)
result = extract_symbols(spdsym)

if result:
    print("Symbol 1:", result[0])  # Output: TITANJAN
    print("Symbol 2:", result[1])  # Output: TITANFEB
else:
    print("No valid symbols found")


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
Name: Description, dtype: object


TypeError: expected string or bytes-like object, got 'Series'

In [25]:
df["displayName"].unique()


array([nan, 'TITAN', 'JSWSTEEL', 'TECHM', 'WIPRO', 'ADANIENT',
       'ULTRACEMCO', 'ITC', 'ASIANPAINT', 'AXISBANK'], dtype=object)

In [27]:
asp=df[df["Name"]=="JSWSTEEL"]
asp

,Unnamed: 0,ExchangeSegment,ExchangeInstrumentID,Name,Description,Series,NameWithSeries,InstrumentID,PriceBand.High,PriceBand.Low,FreezeQty,TickSize,LotSize,Multiplier,displayName,ExpiryDate,Strike,OptionType,TickerName
9,9,NSEFO,10056847,JSWSTEEL,JSWSTEEL25FEB25MARFUT,FUTSTK,JSWSTEEL-FUTSTK,2100110056847,22.65,-22.65,27001,0.05,675,1,NaN,2025-02-27 14:30:00,JSWSTEEL 27FEB27MAR SPD,1,1
17,17,NSEFO,10559186,JSWSTEEL,JSWSTEEL25JAN25FEBFUT,FUTSTK,JSWSTEEL-FUTSTK,2100110559186,22.55,-22.55,27001,0.05,675,1,NaN,2025-01-30 14:30:00,JSWSTEEL 30JAN27FEB SPD,1,1
19,19,NSEFO,41615,JSWSTEEL,JSWSTEEL25MARFUT,FUTSTK,JSWSTEEL-FUTSTK,2508600041615,1003.30,820.85,27001,0.05,675,1,JSWSTEEL,2025-03-27 14:30:00,JSWSTEEL 27MAR2025,1,1
40,40,NSEFO,39122,JSWSTEEL,JSWSTEEL25FEBFUT,FUTSTK,JSWSTEEL-FUTSTK,2505800039122,997.00,815.75,27001,0.05,675,1,JSWSTEEL,2025-02-27 14:30:00,JSWSTEEL 27FEB2025,1,1
43,43,NSEFO,41094,JSWSTEEL,JSWSTEEL25JANFUT,FUTSTK,JSWSTEEL-FUTSTK,2503000041094,990.95,810.80,27001,0.05,675,1,JSWSTEEL,2025-01-30 14:30:00,JSWSTEEL 30JAN2025,1,1


In [ ]:
{10559186:[41094,39122]}

In [49]:
df["InstrumentID"].unique()
df["Name"].unique()


instrument_list = []

df = df.drop_duplicates(subset=['InstrumentID'])
asp = asp.reset_index(drop=True)
for i in range(len(asp["InstrumentID"])):

    instrument_list.append({"exchangeSegment":2,"exchangeInstrumentID":int(asp["ExchangeInstrumentID"][i])})
#save into json file
import json
with open('Titaninstrument_list.json', 'w') as f:
    json.dump(instrument_list, f)


In [51]:
df["InstrumentID"].unique()
df["Name"].unique()


instrument_list = []

df = df.drop_duplicates(subset=['InstrumentID'])
asp = asp.reset_index(drop=True)


#sort asp by expiry date
asp=asp.sort_values(by="ExpiryDate")

for i in range(len(asp["InstrumentID"])):

    instrument_list.append({"exchangeSegment":2,"exchangeInstrumentID":int(asp["ExchangeInstrumentID"][i])})
#save into json file
import json
with open('Titaninstrument_list.json', 'w') as f:
    json.dump(instrument_list, f)


In [47]:
asp

,Unnamed: 0,ExchangeSegment,ExchangeInstrumentID,Name,Description,Series,NameWithSeries,InstrumentID,PriceBand.High,PriceBand.Low,FreezeQty,TickSize,LotSize,Multiplier,displayName,ExpiryDate,Strike,OptionType,TickerName
0,18,NSEFO,42541,TITAN,TITAN25JANFUT,FUTSTK,TITAN-FUTSTK,2503000042541,3796.50,3106.25,7001,0.05,175,1,TITAN,2025-01-30 14:30:00,TITAN 30JAN2025,PE,1
1,31,NSEFO,43400,TITAN,TITAN25MARFUT,FUTSTK,TITAN-FUTSTK,2508600043400,3837.85,3140.10,7001,0.05,175,1,TITAN,2025-03-27 14:30:00,TITAN 27MAR2025,PE,1
2,33,NSEFO,41498,TITAN,TITAN25FEBFUT,FUTSTK,TITAN-FUTSTK,2505800041498,3821.15,3126.40,7001,0.05,175,1,TITAN,2025-02-27 14:30:00,TITAN 27FEB2025,PE,1
